In [ ]:
from google.colab import drive
drive.mount("/content/drive")

%cd /content/drive/MyDrive/local_speech/others/TRANSCRIPTION_DATA

In [ ]:
!pip install sqlalchemy

In [ ]:
from sqlalchemy import create_engine, select, func, text, update, delete
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker
import pandas as pd

from IPython.display import display_html

# Database connection

In [ ]:
db_engine = create_engine("sqlite:////content/drive/MyDrive/local_speech/others/TRANSCRIPTION_DATA/Metadata.db")
Session = sessionmaker(db_engine)
session = Session()

Base = automap_base()
Base.prepare(db_engine)

LabelboxLog = Base.classes.labelbox_log
ReviewSheetLog = Base.classes.reviewsheet_log
ProjectCursor = Base.classes.cursor

# Functions

In [ ]:
REVIEW_SHEET_COLUMNS = """reviewsheet_log.task_id,
    reviewsheet_log.sheetname,
    reviewsheet_log.prev_count,
    reviewsheet_log.curr_count,
    reviewsheet_log.task_type"""


def display_side_by_side(dfs, titles):
    html_str = ""
    for df, title in zip(dfs, titles):
        html_str += f"<div style='margin: 10px;'><h3 style='text-align: center;'>{title}</h3>{df.to_html(index=False)}</div>"
    display_html(f"<div style='display: flex; justify-content: space-around;'>{html_str}</div>", raw=True)

# View info

## Search by District


In [ ]:
district = "Cumilla"

In [ ]:
labelbox_df = pd.read_sql(f"SELECT * FROM labelbox_log WHERE labelbox_log.district = '{district}'", db_engine)
reviewsheet_df = pd.read_sql_query(f"""
    SELECT {REVIEW_SHEET_COLUMNS}
    FROM reviewsheet_log JOIN labelbox_log
    ON reviewsheet_log.task_id = labelbox_log.task_id
    WHERE labelbox_log.district = '{district}';""",
db_engine)

overview = pd.DataFrame((
    ("District", district),
    ("Sheet Created", labelbox_df.date.min()),
    ("Syncs done", labelbox_df.query("date > date.min()").shape[0]),
), columns=("", ""))

display_side_by_side([overview, labelbox_df, reviewsheet_df], ["Overview", "Labelbox", "Reviewsheet", "Overview"])

## Search by ID

In [ ]:
id = "3f2e173f-9de3-48ac-a0a5-767d3ff8638c"

In [ ]:
labelbox_df = pd.read_sql(f"SELECT * FROM labelbox_log WHERE labelbox_log.task_id = '{id}'", db_engine)
reviewsheet_df = pd.read_sql_query(f" SELECT * FROM reviewsheet_log WHERE reviewsheet_log.task_id = '{id}';", db_engine)

overview = pd.DataFrame((
    ("District", district),
    ("Sheet Created", labelbox_df.date.min()),
    ("Syncs done", labelbox_df.query("date > date.min()").shape[0]),
), columns=("", ""))

display_side_by_side([overview, labelbox_df, reviewsheet_df], ["Overview", "Labelbox", "Reviewsheet", "Overview"])

## Newest Version

In [ ]:
pd.read_sql_query(f"""
WITH
filtered_table AS (
    SELECT task_id, max(date) as date FROM labelbox_log
    WHERE district = '{district}'
)

SELECT reviewsheet_log.task_id,
    filtered_table.date,
    reviewsheet_log.sheetname,
    reviewsheet_log.prev_count,
    reviewsheet_log.curr_count,
    reviewsheet_log.task_type
FROM reviewsheet_log
JOIN filtered_table
ON reviewsheet_log.task_id = filtered_table.task_id;
""", db_engine)

## View cursors

In [ ]:
pd.read_sql_table("cursor", db_engine)

## Districts tracked

In [ ]:
pd.Series(
    pd.read_sql_table("labelbox_log", db_engine).district.unique(),
    name="District"
)